## Reader Part

In [1]:
from __future__ import division
import tensorflow as tf
# from tensorflow.contrib import learn -> deprecated
import numpy as np
import os
import time
import datetime
import argparse

In [2]:
def _read_data(filename):
    with open(filename) as fp:
        lines = fp.readlines()
        return lines


def _length(sentences):
    sn_length = [len(sn.split()) for sn in sentences] # get the length of each sentence
    return sn_length

def _mask(sentences, max_length):
    """
    - Builds a mask array to ignore padded integers for calculating precision, recall and fscore

    Args:
         sentences: a list of input sentences
         max_length: maximum length used for padding sentences

    Returns:
        mask_array: an array of actual length of sentences

    """
    sn_length = _length(sentences)
    mask_array = np.zeros((len(sn_length) * max_length, 1), dtype=np.float64) # initialize mask array by zeros, (len(sn_length) * max_length, 1 means the number of all words in all sentences)
    row_num = 0
    for length in sn_length:
        mask_array[row_num:length+row_num] = 1 # set 1 for actual length of sentences by row_num:length+row_num
        row_num += length + (max_length - length) # update row_num by adding length of sentence and padding length
    return mask_array

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import numpy as np

def swbd_data(data_path=None):
    train_input_data = _read_data(os.path.join(data_path, "swbd.train.txt"))
    dev_input_data = _read_data(os.path.join(data_path, "swbd.dev.txt"))
    test_input_data = _read_data(os.path.join(data_path, "swbd.test.txt"))

    max_length = max(_length(train_input_data))

    # Instantiate the tokenizer
    tokenizer = Tokenizer()

    # Fit the tokenizer on your text
    tokenizer.fit_on_texts(train_input_data)

    # Convert text to sequences
    train_input_ids = np.array(tokenizer.texts_to_sequences(train_input_data))
    dev_input_ids = np.array(tokenizer.texts_to_sequences(dev_input_data))
    test_input_ids = np.array(tokenizer.texts_to_sequences(test_input_data))

    train_output_data = _read_data(os.path.join(data_path, "swbd.train.label.txt"))
    dev_output_data = _read_data(os.path.join(data_path, "swbd.dev.label.txt"))
    test_output_data = _read_data(os.path.join(data_path, "swbd.test.label.txt"))

    # For output_vocab_processor, since it's a simple mapping, we can replace it with a manual transformation
    label_vocab = {'F': 0, 'E': 1}
    train_output_ids = np.array([[label_vocab[word] for word in sentence.split()] for sentence in train_output_data])
    dev_output_ids = np.array([[label_vocab[word] for word in sentence.split()] for sentence in dev_output_data])
    test_output_ids = np.array([[label_vocab[word] for word in sentence.split()] for sentence in test_output_data])

    train_mask = _mask(train_input_data, max_length)
    dev_mask = _mask(dev_input_data, max_length)
    test_mask = _mask(test_input_data, max_length)

    return train_input_ids, \
        dev_input_ids, \
        test_input_ids, \
        train_output_ids, \
        dev_output_ids, \
        test_output_ids, \
        train_mask, \
        dev_mask, \
        test_mask, \
        max_length, \
        tokenizer

In [4]:
# function to iterate on the input and output files, and create minibatches of input, output and mask.
def swbd_minibatches(input_ids, output_ids, mask_data, batch_size, num_epochs, max_length, shuffle=True):
    """
    - Iterates on the Switchboard input and output files

    Args:
        input_ids: one of the input id files from swbd_data
        output_ids: one of the output id files from swbd_data
        mask_data: one of the mask files from swbd_data
        batch_size: int, the batch size
        num_epochs: int, the number of training epochs
        max_length: int, the maximum length used for padding
        shuffle: Boolean, whether to shuffle training data or not

    Returns:
        tuple (x, y, z): which are minibathes of (input, output, mask)
    """

    output_ids = np.reshape(np.array(output_ids), (-1, max_length))
    mask_data = np.reshape(np.array(mask_data), (-1, max_length))

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(len(input_ids)))
        input_ids = input_ids[shuffle_indices]
        output_ids = output_ids[shuffle_indices]
        mask_data = mask_data[shuffle_indices]

    input_ids = np.array([np.concatenate(input_ids, 0)]).T
    output_ids = np.array([np.concatenate(output_ids, 0)]).T
    mask_data = mask_data.reshape(-1, 1)

    data_size = len(input_ids) // max_length
    num_batches_per_epoch = data_size // batch_size
    for epoch in range(num_epochs):
        for batch_num in range(num_batches_per_epoch):
            start_index = (batch_num * batch_size) * max_length
            end_index = (min((batch_num + 1) * batch_size, data_size)) * max_length
            x = np.reshape(input_ids[start_index:end_index], (batch_size, max_length))
            y = output_ids[start_index:end_index]
            z = mask_data[start_index:end_index]
            yield (x, y, z)
            
            
def batch_iter(input_id, max_length, mask):
    """
    - Iterates on input data (usef for prediction)

    Args:
        input_id: list of input sentences mapped to integers
        max_length: maximum length of sentences
        mask: list of actual length of sentences
     
    Returns:
        tuple (x_input, z_mask): which are minibathes of (input, mask)
    """

    x = np.array(input_id)
    for sn in range(len(input_id)):
        start = sn * max_length
        end = (1 + sn) * max_length
        x_input = x[sn : sn + 1]
        z_mask = mask[start:end]
        yield (x_input, z_mask)

--- End of Reader ---

## ACNN Implementation

In [7]:
from tensorflow.keras.layers import Input, Dropout, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.losses import BinaryCrossentropy


class ACNN(object):
    """

    An ACNN for disfluency detection:
    which includes an embedding layer, followed by a drop-out layer, 
    an auto-correlational layer, two convolutional layers and a sigmoid layer.

    """
    def __init__(self, max_length, num_classes, vocab_size, embedding_size, 
                 conv1_filter_sizes, conv2_filter_sizes, conv3_filter_sizes, 
                 num_filters, embed_initial, weight_initial, device_name):
        
        # Inputs
        self.input_x = Input(shape=(max_length,), dtype='int32', name="input_x")
        self.input_y = Input(shape=(num_classes,), dtype='float32', name="input_y")
        self.input_z = Input(shape=(1,), dtype='float32', name="input_z")
        self.dropout_keep_prob = tf.Variable(initial_value=0.5, trainable=False, name="dropout_keep_prob")
        self.l2_reg_lambda = tf.Variable(initial_value=0.0, trainable=False, name="l2_reg_lambda")
        self.batch_size = tf.Variable(initial_value=32, trainable=False, shape=[], name="batch_size")

        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #                        Embedding Layer
        # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        self.embedding = Embedding(input_dim=vocab_size, 
                                   output_dim=embedding_size, 
                                   embeddings_initializer='uniform',
                                   input_length=max_length,
                                   name="embedding")
        self.embedded_words = self.embedding(self.input_x)
        self.embedded_words_expanded = tf.expand_dims(self.embedded_words, -1)

        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #                        Drop-out Layer
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

        self.dropout = Dropout(rate=1-self.dropout_keep_prob.numpy())
        self.h_drop = self.dropout(self.embedded_words_expanded)
        

        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #                        Auto-Correlation Layer
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        """
        The auto-correlation layer includes two parts:
            1. An auto-correlated tensor which is constructed by comparing each input vector u 
            to the input vector v using a binary function f. 
            The auto-correlated tensor is then convolved with 3D or 4D kernels B of different sizes.

            2. A vanilla CNN layer which convolves the input tensor with kernels A of different sizes.
            
            Each kernel group A and B outputs a tensor of the same size which are added element-wise 
            to produce the feature representation that is passed to further convolutional layers. 
            For more details, read https://www.aclweb.org/anthology/D18-1490.pdf.
        """
        self.combine_acnn_features = []
        for i, filter_size in enumerate(conv1_filter_sizes):
            # First part of the ACNN layer:
            kernel_B_shape = [conv1_filter_sizes[i], conv1_filter_sizes[i], num_filters] 
            kernel_B = tf.Variable(tf.random.truncated_normal(kernel_B_shape, stddev=weight_initial), name="w")
            flat_kernel_B = tf.reshape(kernel_B, (-1, num_filters))
            patches = tf.image.extract_patches(self.h_drop,
                                               sizes=[1, conv1_filter_sizes[i], embedding_size, 1],
                                               strides=[1, 1, embedding_size, 1],
                                               rates=[1, 1, 1, 1],
                                               padding="SAME",
                                               name="patches"
                                               )
            reshaped_patches = tf.reshape(patches, [-1, conv1_filter_sizes[i], embedding_size])
            function_f = tf.einsum('ijl,ikl->ijk', reshaped_patches, reshaped_patches) 
            reshaped_function_f = tf.reshape(function_f, [self.batch_size.numpy() * max_length, -1])
            auto_correlated_input = tf.reshape(tf.matmul(
                                               reshaped_function_f, flat_kernel_B, name='auto_cor'
                                               ), (self.batch_size.numpy(), max_length, 1, -1))

            # Second part of the ACNN layer:
            conv1 = Conv2D(filters=num_filters, 
                           kernel_size=(conv1_filter_sizes[i], embedding_size), 
                           strides=(1, embedding_size), 
                           padding='same', 
                           activation='relu', 
                           name="conv1")
            convolved_input = conv1(self.h_drop)

            # Element-wise addition of the outputs of 1st and 2nd parts of the ACNN layer:
            added_outputs = tf.add(auto_correlated_input, convolved_input)
            auto_correlation = tf.nn.relu(added_outputs)
            self.combine_acnn_features.append(auto_correlation)

        # Combine all the ACNN features:
        num_filters_total = num_filters * len(conv1_filter_sizes)
        self.all_acnn_features = tf.concat(self.combine_acnn_features, 3)

        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #                        Convolutional Layer
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        self.combine_conv3_features = []
        conv3_input = tf.reshape(self.all_acnn_features, [-1, max_length, num_filters_total, 1])
        for k, filter_size in enumerate(conv3_filter_sizes):
            conv3 = Conv2D(filters=num_filters, 
                           kernel_size=(conv3_filter_sizes[k], num_filters_total), 
                           strides=(1, num_filters_total), 
                           padding='same', 
                           activation='relu', 
                           name="conv3")
            conv3_output = conv3(conv3_input)
            self.combine_conv3_features.append(conv3_output)

        # Combine all the conv3 features:
        num_filters_total = num_filters * len(conv3_filter_sizes)
        self.all_conv3_features = tf.concat(self.combine_conv3_features, 3)
        self.reshaped_conv3_features = tf.reshape(self.all_conv3_features, [-1, num_filters_total])

        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #                         1-Width Convolution
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        # Fully Connected Layer
        self.l2_loss = tf.constant(0.0) # keeping track of l2 regularization loss
        self.local4 = Dense(units=128, 
                            kernel_initializer='glorot_uniform', 
                            activation='relu', 
                            name="local4")
        local4_output = self.local4(self.reshaped_conv3_features)
        self.l2_loss += tf.nn.l2_loss(self.local4.weights[0]) # weights[0] should give the kernel weights

        # Final scores and predictions
        self.output = Dense(units=num_classes, 
                            kernel_initializer='glorot_uniform', 
                            activation='sigmoid', 
                            name="output")
        self.scores = self.output(local4_output)
        self.l2_loss += tf.nn.l2_loss(self.output.weights[0]) # weights[0] should give the kernel weights
        self.probs = tf.sigmoid(self.scores, name="sigmoid") 
        condition = tf.less(self.probs, tf.fill(tf.shape(self.probs), 0.5))
        self.predictions = tf.where(
            condition, tf.zeros(tf.shape(self.probs)), tf.ones(tf.shape(self.probs))
            ) # if prob >= 0.5 ==> 1 (i.e. disfluent); else ==> 0 (i.e. fluent)


        # Calculate sigmoid cross entropy loss
        self.loss_fn = BinaryCrossentropy(from_logits=True)
        losses = self.loss_fn(self.input_y, self.scores)
        masked_losses = tf.transpose(self.input_z) * losses
        self.loss = (tf.reduce_sum(masked_losses) / tf.cast(self.batch_size, "float32")) \
                    + (self.l2_reg_lambda * self.l2_loss)

        # Calculate f-score
        fscore_mask = tf.cast(self.input_z, "int64")
        predictions = tf.cast(self.predictions, "int64")
        input_y = tf.cast(self.input_y, "int64")

        # e ==> num of edited predictions
        masked_prediction = fscore_mask * predictions
        e = tf.reduce_sum(masked_prediction)
        self.nprediction = tf.cast(e, dtype=tf.int32, name="nprediction")

        # g ==> num of edited words in gold set
        masked_input_y = fscore_mask * input_y
        g = tf.reduce_sum(masked_input_y)

## End of ACNN class

## Train Module